## Using MoSH to fit body models to labeled mocaps

We base this off of the two SOMA tutorials: [Run SOMA on SOMA dataset](https://github.com/nghorbani/soma/blob/main/src/tutorials/run_soma_on_soma_dataset.ipynb) and [Solve Already Labeled MoCaps with Mosh++](https://github.com/nghorbani/soma/blob/main/src/tutorials/solve_labeled_mocap.ipynb).

Reading the [definitions](https://github.com/nghorbani/soma/tree/main/src/tutorials#definitions) of terminology is recommended.

MoSH++ takes in labeled motion capture data and fits an SMPL body model to it. As a note, the original MoSH paper converted the motion capture data to a different body model; we use MoSH/MoSH++ interchangeably, with both referring to the MoSH++.

A *labeled* motion capture is simply one that has markers assigned to the point cloud points for each frame. The exact names of the markers is not relevant, just their stability (Is the marker mapped to the same point consistently for each frame)  (TODO: verify this).

From the [MoSH paper](https://files.is.tue.mpg.de/black/papers/MoSh.pdf):

> Marker placement on the human body varies across subjects and sessions, consequently we do not assume that the exact marker placement is known. Instead, a key contribution of MoSh is that it solves for the observed marker locations relative to the 3D body model.

The MoSH algorithm works in two steps:

1. Solving for Beta Parameters. A single frame from the sequence is picked, and the SMPL body model is fit to the frame. SMPL models, in their simplest form, have 82 parameters per frame: 23 joints, 3 parameters to represent each joint in 3d space with axis-angle rotation, 3 more parameters to represent the root of the body (global translation), and 10 Beta parameters to characterize the variation of body height, body proportion, and weight. Once these Beta Parameters are solved for, they are fixed, and are used to fit the rest of the frames in the sequence. This process can take 20-30minutes per subject.
2. With Betas fixed, MoSH++ fits the SMPL body model frame by frame for the sequence. This process takes ~2 seconds per frame.

TODO: cite the paper that has the approximate times for each step.

TODO: bring in info about how MoSH uses temporal smoothing for its frame-by-frame fitting.


### Verify Environment

In [2]:
# If this doesn't work, something is wrong with the conda environment. Check conda-env.md
# Make sure you are using the right kernel
import soma
import psbody
import moshpp

In [3]:
from glob import glob
import os
import os.path as osp
import time

from loguru import logger

from soma.amass.mosh_manual import mosh_manual

## Running MoSH

First, we define the directories we will be using to find our mocaps, and where to store them as well.

In [4]:
soma_work_base_dir = osp.join(os.getcwd(), 'soma-root')
support_base_dir = osp.join(soma_work_base_dir, 'support_files')

mocap_base_dir = osp.join(support_base_dir, 'evaluation_mocaps/original')

work_base_dir = osp.join(soma_work_base_dir, 'running_just_mosh')
temp_base_dir = osp.join(support_base_dir, 'render_out_temp')

# list each of the dataset folders you want to process with MoSH
# Verify that the location of these folders is at TODO: fill this in
# For raw, unlabeled mocaps, make sure to run the labeling script first at `label.ipynb`
# Run on 'immersion_test' to see sample results on a few mocaps
target_ds_names = ['SOMA_manual_labeled',]

### View list of Mocaps to MoSH

In [5]:
def select(fname):
    '''
    Running MoSH takes time, so a filter can be applied if only a subset of mocaps is needed.
    '''
    # add filters here if needed
    return True

In [9]:
# List out all filenames we want to MoSH.
for ds_name in target_ds_names:
    mocap_fnames = glob(osp.join(mocap_base_dir, ds_name,  '*/*.c3d'))

    mocap_fnames = list(filter(select, mocap_fnames)) # by default, `select()` does not filter anything
    logger.info(f'#mocaps found for {ds_name}: {len(mocap_fnames)}')
    logger.info(f'mocaps:\t{mocap_fnames}...')

2023-04-10 13:56:56.773 | INFO     | __main__:<module>:6 - #mocaps found for SOMA_manual_labeled: 69
2023-04-10 13:56:56.775 | INFO     | __main__:<module>:7 - mocaps:	['/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/SOMA_manual_labeled/soma_subject2/sit_003.c3d', '/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/SOMA_manual_labeled/soma_subject2/clap_002.c3d', '/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/SOMA_manual_labeled/soma_subject2/squat_002.c3d', '/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/SOMA_manual_labeled/soma_subject2/lift_002.c3d', '/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/SOMA_manual_labeled/soma_subject2/dance_003.c3d', '/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/SOMA_manual_lab

In [10]:
for ds_name in target_ds_names:
    mocap_fnames = glob(osp.join(mocap_base_dir, ds_name,  '*/*.c3d'))

    mocap_fnames = list(filter(select, mocap_fnames))

    logger.info(f'#mocaps found for {ds_name}: {len(mocap_fnames)}')

    mosh_manual(
        mocap_fnames,
        mosh_cfg={
            'moshpp.verbosity': 1, # set to 2 to visualize the process in meshviewer
            'dirs.work_base_dir': osp.join(work_base_dir, 'mosh_results'),
            'dirs.support_base_dir': support_base_dir,
            'mosh_stagei_perseq': True,
        },
        parallel_cfg={
            'pool_size': 5,
            'max_num_jobs': 1,
            'randomly_run_jobs': True,
        },
        run_tasks=[
            'mosh',
        ],
        fast_dev_run=False,
    )

2023-04-10 13:57:16.485 | INFO     | __main__:<module>:6 - #mocaps found for SOMA_manual_labeled: 69
2023-04-10 13:57:25.789 | INFO     | soma.amass.mosh_manual:mosh_manual:109 - Submitting MoSh++ jobs.
2023-04-10 13:57:25.799 | INFO     | soma.tools.parallel_tools:run_parallel_jobs:54 - #Job(s) submitted: 2
2023-04-10 13:57:25.801 | INFO     | soma.tools.parallel_tools:run_parallel_jobs:58 - max_num_jobs is set to 1. choosing the first #Job(s): 1
2023-04-10 13:57:25.802 | INFO     | soma.tools.parallel_tools:run_parallel_jobs:67 - Will run the jobs in random order.


soma_subject2 -- sit_003 -- mosh_head:__init__:95 -- mocap_fname: /home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/SOMA_manual_labeled/soma_subject2/sit_003.c3d
soma_subject2 -- sit_003 -- mosh_head:__init__:97 -- stagei_fname: /home/ritaank/Documents/dev/SOMA-interface/soma-root/running_just_mosh/mosh_results/SOMA_manual_labeled/soma_subject2/male_stagei.pkl
soma_subject2 -- sit_003 -- mosh_head:__init__:98 -- stageii_fname: /home/ritaank/Documents/dev/SOMA-interface/soma-root/running_just_mosh/mosh_results/SOMA_manual_labeled/soma_subject2/sit_003_stageii.pkl
soma_subject2 -- sit_003 -- mosh_head:__init__:103 -- surface_model: type: smplx; gender: male; fname:/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/smplx/male/model.pkl
soma_subject2 -- sit_003 -- mosh_head:__init__:107 -- optimize_fingers: False, optimize_face: False, optimize_toes: False, optimize_betas: True, optimize_dynamics: False
soma_subject2 -- sit_003 --

/home/ritaank/miniconda3/envs/soma3/lib/python3.7/site-packages/omegaconf/resolvers/oc/__init__.py:91: UserWarning: 'dirs.markerlyout_basename' is deprecated. Change your code and config to use 'dirs.marker_layout.basename'
  warnings.warn(category=UserWarning, message=msg)
/home/ritaank/miniconda3/envs/soma3/lib/python3.7/site-packages/omegaconf/resolvers/oc/__init__.py:91: UserWarning: 'dirs.marker_layout_fname' is deprecated. Change your code and config to use 'dirs.marker_layout.fname'
  warnings.warn(category=UserWarning, message=msg)


soma_subject2 -- sit_003 -- edit_tools:marker_layout_load:130 -- Loading marker layout: /home/ritaank/Documents/dev/SOMA-interface/soma-root/running_just_mosh/mosh_results/SOMA_manual_labeled/soma_subject2/male_stagei.json
soma_subject2 -- sit_003 -- edit_tools:marker_layout_load:134 -- Available marker types: {'body': 53}. Total: 53 markers.
soma_subject2 -- sit_003 -- edit_tools:marker_layout_load:130 -- Loading marker layout: /home/ritaank/Documents/dev/SOMA-interface/soma-root/running_just_mosh/mosh_results/SOMA_manual_labeled/soma_subject2/male_stagei.json
soma_subject2 -- sit_003 -- edit_tools:marker_layout_load:134 -- Available marker types: {'body': 53}. Total: 53 markers.
soma_subject2 -- sit_003 -- mosh_head:dump_stagei_marker_layout:338 -- created /home/ritaank/Documents/dev/SOMA-interface/soma-root/running_just_mosh/mosh_results/SOMA_manual_labeled/soma_subject2/male_stagei.json
soma_subject2 -- sit_003 -- mosh_head:dump_stagei_marker_layout:339 -- created /home/ritaank/Doc

/home/ritaank/miniconda3/envs/soma3/lib/python3.7/site-packages/omegaconf/resolvers/oc/__init__.py:91: UserWarning: 'dirs.markerlyout_basename' is deprecated. Change your code and config to use 'dirs.marker_layout.basename'
  warnings.warn(category=UserWarning, message=msg)
/home/ritaank/miniconda3/envs/soma3/lib/python3.7/site-packages/omegaconf/resolvers/oc/__init__.py:91: UserWarning: 'dirs.marker_layout_fname' is deprecated. Change your code and config to use 'dirs.marker_layout.fname'
  warnings.warn(category=UserWarning, message=msg)


soma_subject2 -- sit_003 -- mosh_head:mosh_stageii:298 -- finished mosh stageii in 0:41:02.114942
soma_subject2 -- sit_003 -- mosh_head:run_moshpp_once:606 -- Final mosh stageii loss: data = 8.79e+08 | poseB = 4.79e+08 | velo = 1.61e+01


## Rendering outputs

After our files have been MoSHed, we can render the output SMPL mesh as an mp4, a series of pngs, and/or a blender file.
We repeat a similar process to above, changing the `run_task` to render, as well as defining different configuration variables. We also need to define a `temp_base_dir` to store the intermediate rendering files.

In [6]:
for ds_name in target_ds_names:
    mocap_fnames = glob(osp.join(mocap_base_dir, ds_name,  '*/*.c3d'))

    mocap_fnames = list(filter(select, mocap_fnames))

    logger.info(f'#mocaps found for {ds_name}: {len(mocap_fnames)}')

    mosh_manual(
        mocap_fnames,
        mosh_cfg={
            # 'moshpp.verbosity': 1, # set to 2 to visualize the process in meshviewer
            'dirs.work_base_dir': osp.join(work_base_dir, 'mosh_results'),
            'dirs.support_base_dir': support_base_dir,
            # 'mosh_stagei_perseq': True,
        },
        render_cfg={
            # 'render.render_only_one_image': True, # uncomment for initial testing of the pipeline
            'render.video_fps': 60,  # video_fps * ds_rate should equal the sample fps always
            'mesh.ds_rate': 4, # see above
            'render.resolution.change_from_blend': True,
            'render.resolution.default': [1600, 1600],  # [x,y]
            'render.render_engine': 'eevee',  # eevee / cycles,
        
            'dirs.work_base_dir': osp.join(work_base_dir, 'mp4_renders'),
            # 'render.show_markers': True,
            'render.save_final_blend_file': True,
            'dirs.support_base_dir': support_base_dir,
            'dirs.temp_base_dir': temp_base_dir
        },
        parallel_cfg={
            'pool_size': 5,
            'max_num_jobs': 2,
            'randomly_run_jobs': True,
        },
        run_tasks=[
            'render',
        ],
        fast_dev_run=False,
    )

2023-04-10 20:37:58.386 | INFO     | __main__:<module>:6 - #mocaps found for SOMA_manual_labeled: 69
2023-04-10 20:38:08.950 | INFO     | soma.amass.mosh_manual:mosh_manual:116 - Submitting render jobs.
2023-04-10 20:38:08.959 | INFO     | soma.tools.parallel_tools:run_parallel_jobs:54 - #Job(s) submitted: 1
2023-04-10 20:38:08.960 | INFO     | soma.tools.parallel_tools:run_parallel_jobs:58 - max_num_jobs is set to 2. choosing the first #Job(s): 1
2023-04-10 20:38:08.961 | INFO     | soma.tools.parallel_tools:run_parallel_jobs:67 - Will run the jobs in random order.
2023-04-10 20:38:09.723 | INFO     | soma.render.parameters_to_mesh:convert_to_mesh_once:67 - Preparing mesh files for: ['/home/ritaank/Documents/dev/SOMA-interface/soma-root/running_just_mosh/mosh_results/SOMA_manual_labeled/soma_subject2/sit_003_stageii.pkl']
2023-04-10 20:38:09.728 | INFO     | soma.render.parameters_to_mesh:convert_to_mesh_once:68 - dirs.mesh_out_dir: /home/ritaank/Documents/dev/SOMA-interface/soma-root

Read blend: /home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/blender/blend_files/soma_standard.blend
Info: Deleted 4 object(s)
(  0.0000 sec |   0.0000 sec) Importing OBJ '/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/render_out_temp/SOMA_manual_labeled/mesh_files/soma_subject2_sit_003/body_mesh/00008.obj'...
  (  0.0001 sec |   0.0000 sec) Parsing OBJ file...
    (  0.2024 sec |   0.2023 sec) Done, loading materials and images...
    (  0.2040 sec |   0.2039 sec) Done, building geometries (verts:10475 faces:20908 materials: 1 smoothgroups:0) ...
    (  0.2655 sec |   0.2654 sec) Done.
  (  0.2656 sec |   0.2655 sec) Finished importing: '/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/render_out_temp/SOMA_manual_labeled/mesh_files/soma_subject2_sit_003/body_mesh/00008.obj'
Progress: 100.00%



2023-04-10 20:38:47.168 | SUCCESS  | soma.render.mesh_to_video_standard:run_blender_once:135 - created /home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/render_out_temp/SOMA_manual_labeled/png_files/soma_standard/soma_subject2_sit_003/00008.png


Info: Saved "00008.blend"
Info: Deleted 2 object(s)
Info: Deleted 0 object(s)
(  0.0000 sec |   0.0000 sec) Importing OBJ '/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/render_out_temp/SOMA_manual_labeled/mesh_files/soma_subject2_sit_003/body_mesh/00012.obj'...
  (  0.0001 sec |   0.0001 sec) Parsing OBJ file...
    (  0.1771 sec |   0.1770 sec) Done, loading materials and images...
    (  0.1775 sec |   0.1774 sec) Done, building geometries (verts:10475 faces:20908 materials: 1 smoothgroups:0) ...
    (  0.2405 sec |   0.2404 sec) Done.
  (  0.2407 sec |   0.2407 sec) Finished importing: '/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/render_out_temp/SOMA_manual_labeled/mesh_files/soma_subject2_sit_003/body_mesh/00012.obj'
Progress: 100.00%



2023-04-10 20:38:48.775 | SUCCESS  | soma.render.mesh_to_video_standard:run_blender_once:135 - created /home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/render_out_temp/SOMA_manual_labeled/png_files/soma_standard/soma_subject2_sit_003/00012.png


...

Info: Saved "00048.blend"
Info: Deleted 2 object(s)
Info: Deleted 0 object(s)
(  0.0000 sec |   0.0000 sec) Importing OBJ '/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/render_out_temp/SOMA_manual_labeled/mesh_files/soma_subject2_sit_003/body_mesh/00052.obj'...
  (  0.0001 sec |   0.0001 sec) Parsing OBJ file...
    (  0.1960 sec |   0.1958 sec) Done, loading materials and images...
    (  0.1966 sec |   0.1964 sec) Done, building geometries (verts:10475 faces:20908 materials: 1 smoothgroups:0) ...
    (  0.2599 sec |   0.2598 sec) Done.
  (  0.2600 sec |   0.2600 sec) Finished importing: '/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/render_out_temp/SOMA_manual_labeled/mesh_files/soma_subject2_sit_003/body_mesh/00052.obj'
Progress: 100.00%



2023-04-10 20:39:04.556 | SUCCESS  | soma.render.mesh_to_video_standard:run_blender_once:135 - created /home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/render_out_temp/SOMA_manual_labeled/png_files/soma_standard/soma_subject2_sit_003/00052.png


Info: Saved "00052.blend"
Info: Deleted 2 object(s)


: 

: 